In [1]:
import os
os.chdir("../")

In [2]:
import nltk

# Download the 'punkt' tokenizer
# nltk.download('punkt')

for pkg in ['punkt', 'averaged_perceptron_tagger']:
    nltk.download(pkg)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
from typing import List
from langchain.document_loaders.unstructured import UnstructuredFileLoader
import cv2
from PIL import Image
import numpy as np
from configs import PDF_OCR_THRESHOLD
from document_loaders.ocr import get_ocr
import tqdm


class RapidOCRPDFLoader(UnstructuredFileLoader):
    def _get_elements(self) -> List:
        def rotate_img(img, angle):
            '''
            img   --image
            angle --rotation angle
            return--rotated img
            '''
            
            h, w = img.shape[:2]
            rotate_center = (w/2, h/2)
            #获取旋转矩阵
            # 参数1为旋转中心点;
            # 参数2为旋转角度,正值-逆时针旋转;负值-顺时针旋转
            # 参数3为各向同性的比例因子,1.0原图，2.0变成原来的2倍，0.5变成原来的0.5倍
            M = cv2.getRotationMatrix2D(rotate_center, angle, 1.0)
            #计算图像新边界
            new_w = int(h * np.abs(M[0, 1]) + w * np.abs(M[0, 0]))
            new_h = int(h * np.abs(M[0, 0]) + w * np.abs(M[0, 1]))
            #调整旋转矩阵以考虑平移
            M[0, 2] += (new_w - w) / 2
            M[1, 2] += (new_h - h) / 2

            rotated_img = cv2.warpAffine(img, M, (new_w, new_h))
            return rotated_img
        
        def pdf2text(filepath):
            import fitz # pyMuPDF里面的fitz包，不要与pip install fitz混淆
            import numpy as np
            ocr = get_ocr()
            doc = fitz.open(filepath)
            resp = ""

            b_unit = tqdm.tqdm(total=doc.page_count, desc="RapidOCRPDFLoader context page index: 0")
            for i, page in enumerate(doc):
                b_unit.set_description("RapidOCRPDFLoader context page index: {}".format(i))
                b_unit.refresh()
                text = page.get_text("")
                resp += text + "\n"

                img_list = page.get_image_info(xrefs=True)
                for img in img_list:
                    if xref := img.get("xref"):
                        bbox = img["bbox"]
                        # 检查图片尺寸是否超过设定的阈值
                        if ((bbox[2] - bbox[0]) / (page.rect.width) < PDF_OCR_THRESHOLD[0]
                            or (bbox[3] - bbox[1]) / (page.rect.height) < PDF_OCR_THRESHOLD[1]):
                            continue
                        pix = fitz.Pixmap(doc, xref)
                        samples = pix.samples
                        if int(page.rotation)!=0:  #如果Page有旋转角度，则旋转图片
                            img_array = np.frombuffer(pix.samples, dtype=np.uint8).reshape(pix.height, pix.width, -1)
                            tmp_img = Image.fromarray(img_array);
                            ori_img = cv2.cvtColor(np.array(tmp_img),cv2.COLOR_RGB2BGR)
                            rot_img = rotate_img(img=ori_img, angle=360-page.rotation)
                            img_array = cv2.cvtColor(rot_img, cv2.COLOR_RGB2BGR)
                        else:
                            img_array = np.frombuffer(pix.samples, dtype=np.uint8).reshape(pix.height, pix.width, -1)

                        result, _ = ocr(img_array)
                        if result:
                            ocr_result = [line[1] for line in result]
                            resp += "\n".join(ocr_result)

                # 更新进度
                b_unit.update(1)
            return resp

        text = pdf2text(self.file_path)
        print("#")
        print(type(text))
        # 打开文件，模式为写入模式，如果文件不存在会自动创建
        with open("/root/llm/Langchain-Chatchat-master/notebooks/output.txt", "w", encoding="utf-8") as file:
            # 将字符串写入文件
            file.write(text) 
        
        from unstructured.partition.text import partition_text
        return partition_text(text=text, **self.unstructured_kwargs)


# if __name__ == "__main__":
file_path = "/root/llm/Langchain-Chatchat-master/data/厂家产品手册/锐捷/20210910211119_RG-S6000E系列交换机RGOS 11.4(1)B12P32S1版本配置手册(V1.0).pdf"
loader = RapidOCRPDFLoader(file_path)
docs = loader.load()

print("##########")
# print(docs)


RapidOCRPDFLoader context page index: 2744: 100%|██████████| 2745/2745 [01:14<00:00, 36.74it/s]


#
<class 'str'>
##########


In [14]:
docs[0].page_content[-100:-1]

'       lost+found  rgos        sys\n\n【检验方法】\n\n1-8\n\n配置指南\n\n诊断命令\n\n常见错误\n\n\uf03d\n\n同一时刻，只允许一个终端执行诊断命令。\n\n1.5 监视与维'

In [17]:
docs[0].page_content[0:10000]

'配置手册\n\nRG-S6000E 系列交换机\n\nS6000E_RGOS11.4(1)B12P32S1\n\n文档版本 ：V1.0\n\ncopyright © 2021 锐捷网络\n\n版权声明\n\ncopyright © 2021 锐捷网络\n\n保留对本文档及本声明的一切权利。\n\n未得到锐捷网络的书面许可，任何单位和个人不得以任何方式或形式对本文档的部分内容或全部进行复制、摘录、备份、修\n\n改、传播、翻译成其他语言、将其全部或部分用于商业用途。\n\n以上均为锐捷网络的商标。\n\n本文档提及的其他所有商标或注册商标，由各自的所有人拥有。\n\n免责声明\n\n您所购买的产品、服务或特性等应受商业合同和条款的约束，本文档中描述的全部或部分产品、服务或特性可能不在您的购\n\n买或使用范围之内。除非合同另有约定，锐捷网络对本文档内容不做任何明示或默示的声明或保证。\n\n由于产品版本升级或其他原因，本文档内容会不定期进行更新。锐捷网络保留在没有任何通知或者提示的情况下对文档内容\n\n进行修改的权利。\n\n本手册仅作为使用指导。锐捷网络在编写本手册时已尽力保证其内容准确可靠，但并不确保手册内容完全没有错误或遗漏，\n\n本手册中的所有信息也不构成任何明示或暗示的担保。\n\n前言\n\n读者对象\n\n本书适合下列人员阅读\n\n\uf06c\n\n网络工程师\n\n\uf06c\n\n技术推广人员\n\n\uf06c\n\n网络管理员\n\n技术支持\n\n\uf06c\n\n锐捷网络官方网站：http://www.ruijie.com.cn/\n\n\uf06c\n\n锐捷网络在线客服：http://webchat.ruijie.com.cn\n\n\uf06c\n\n锐捷网络官方网站服务与支持版块：http://www.ruijie.com.cn/service.aspx\n\n\uf06c\n\n7×24 小时技术服务热线：4008-111-000\n\n\uf06c\n\n锐捷网络技术论坛：http://bbs.ruijie.com.cn/portal.php\n\n\uf06c\n\n常见问题搜索：http://www.ruijie.com.cn/service/know.aspx\n\n\uf06c\n\n锐捷网络技术支持与反馈信

In [1]:
from langchain.text_splitter import CharacterTextSplitter
import re
from typing import List


class AliTextSplitter(CharacterTextSplitter):
    def __init__(self, pdf: bool = False, **kwargs):
        super().__init__(**kwargs)
        self.pdf = pdf

    def split_text(self, text: str) -> List[str]:
        # use_document_segmentation参数指定是否用语义切分文档，此处采取的文档语义分割模型为达摩院开源的nlp_bert_document-segmentation_chinese-base，论文见https://arxiv.org/abs/2107.09278
        # 如果使用模型进行文档语义切分，那么需要安装modelscope[nlp]：pip install "modelscope[nlp]" -f https://modelscope.oss-cn-beijing.aliyuncs.com/releases/repo.html
        # 考虑到使用了三个模型，可能对于低配置gpu不太友好，因此这里将模型load进cpu计算，有需要的话可以替换device为自己的显卡id
        if self.pdf:
            text = re.sub(r"\n{3,}", r"\n", text)
            text = re.sub('\s', " ", text)
            text = re.sub("\n\n", "", text)
        try:
            from modelscope.pipelines import pipeline
        except ImportError:
            raise ImportError(
                "Could not import modelscope python package. "
                "Please install modelscope with `pip install modelscope`. "
            )


        p = pipeline(
            task="document-segmentation",
            model='/root/model/ali/nlp_bert_document-segmentation_chinese-base',
            device="gpu")
        result = p(documents=text)
        sent_list = [i for i in result["text"].split("\n\t") if i]
        return sent_list


In [2]:
with open ("output-Copy1.txt","r",encoding="utf-8") as f:
    content = f.read()

In [3]:
content[0:10]

' \n配置手册 \nRG'

In [3]:
spliter = AliTextSplitter()
sent_list = spliter.split_text(content)

2024-06-07 14:46:39,665 - modelscope - INFO - PyTorch version 2.1.2 Found.
2024-06-07 14:46:39,820 - modelscope - INFO - Loading ast index from /root/.cache/modelscope/ast_indexer
2024-06-07 14:46:43,290 - modelscope - INFO - Loading done! Current index file version is 1.14.0, with md5 521f86a77a0a91aeebd5bf646b81ffed and a total number of 976 components indexed
/root/.local/conda/envs/chatchat/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-06-07 14:47:36,237 - modelscope - INFO - initiate model from /root/model/ali/nlp_bert_document-segmentation_chinese-base
2024-06-07 14:47:36,239 - modelscope - INFO - initiate model from location /root/model/ali/nlp_bert_document-segmentation_chinese-base.
2024-06-07 14:47:36,241 - modelscope - INFO - initialize model from /root/model/ali/nlp_bert_document-segm

OutOfMemoryError: CUDA out of memory. Tried to allocate 40.39 GiB. GPU 0 has a total capacty of 31.74 GiB of which 605.12 MiB is free. Process 3200058 has 31.14 GiB memory in use. Of the allocated memory 30.73 GiB is allocated by PyTorch, and 49.66 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [4]:
sent_list[0]
# docs[0]

NameError: name 'docs' is not defined

In [21]:
#模型下载
from modelscope import snapshot_download
model_dir = snapshot_download('iic/nlp_bert_document-segmentation_chinese-base')

2024-06-07 11:23:32,622 - modelscope - WARNING - Model revision not specified, use revision: v1.0.1
Downloading: 100%|██████████| 16.0/16.0 [00:00<00:00, 60.2kB/s]
Downloading: 100%|██████████| 1.02k/1.02k [00:00<00:00, 3.74MB/s]
Downloading: 100%|██████████| 296/296 [00:00<00:00, 1.14MB/s]
Downloading: 100%|██████████| 388M/388M [05:39<00:00, 1.20MB/s] 
Downloading: 100%|██████████| 5.94k/5.94k [00:00<00:00, 12.4MB/s]
Downloading: 100%|██████████| 134/134 [00:00<00:00, 781kB/s]
Downloading: 100%|██████████| 263k/263k [00:00<00:00, 1.93MB/s]
Downloading: 100%|██████████| 316/316 [00:00<00:00, 1.74MB/s]
Downloading: 100%|██████████| 107k/107k [00:00<00:00, 1.22MB/s]
Downloading: 100%|██████████| 220k/220k [00:00<00:00, 586kB/s]
Downloading: 100%|██████████| 248k/248k [00:00<00:00, 955kB/s]


In [26]:
from unstructured.partition.pdf import partition_pdf
# elements = partition_pdf("/root/llm/Langchain-Chatchat-master/notebooks/layout-parser-paper.pdf")
elements = partition_pdf("/root/llm/Langchain-Chatchat-master/data/厂家产品手册/锐捷/20210910211119_RG-S6000E系列交换机RGOS 11.4(1)B12P32S1版本配置手册(V1.0).pdf")

2024-06-07 16:35:27,926 - cmapdb.py[line:481] - WARNING: Ignoring (part of) ToUnicode map because the PDF data does not conform to the format. This could result in (cid) values in the output. The start object is not a byte.
2024-06-07 16:37:13,780 - cmapdb.py[line:481] - WARNING: Ignoring (part of) ToUnicode map because the PDF data does not conform to the format. This could result in (cid) values in the output. The start object is not a byte.


In [31]:
str(elements[0])

'配置手册'

In [43]:
print("\n\n".join([str(el) for el in elements[86:94]]))


下图列举了在 PC 上通过 Secure CRT 与网络设备 A 建立连接，并打开命令行界面配置命令。

图 1-2

1-1

命令行界面

配置指南

命令行界面

1.3 功能详解

功能特性


In [2]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("RG-S6000E/20210910211119_RG-S6000E系列交换机RGOS 11.4(1)B12P32S1版本配置手册(V1.0).pdf")
pages = loader.load_and_split()

In [1]:
print("#")

#


In [14]:
pages[301]

Document(page_content='4-6 配置指南  VLAN   \n\uf0e6 向当前 VLAN添加 ACCESS 口 \n\uf06c 可选配置 。 \n\uf06c 该命令只对 Access口有效，VLAN添加 Access口后，接口可传输该 VLAN数据。  \n\uf06c 交换机设备上配置。  \n【命令格式】  add interface  { interface -id | range  interface -range  } \n【参数说明】  interface -id：单个接口  \ninterface -range：多个接口  \n【缺省配置】  缺省情况下，所有二层以太网口都属于 VLAN1  \n【命令模式】  VLAN配置模式  \n【使用指导】  在VLAN配置模式下，将指定的 Access口加入该 VLAN。该命令的配置效果同在接口模式下指定该接口所属\nVLAN的命令（即 switchport  access vlan  vlan-id）效果一致。  \n 对于两种形式的接口加入 VLAN命令，配置生效的原则是后配置的命令覆盖前面配置的命令  \n检验方法  \n\uf06c 往ACCESS 口发送 untag报文，报文在该 VLAN内广播。  \n\uf06c 使用命令 show  vlan和show  interface  switchport 查看配置显示是否生效。  \n【命令格式】  show vlan  [ id vlan-id ] \n【参数说明】  vlan-id ：VLAN ID 号 \n【命令模式】  所有模式  \n【使用指导】  - \n【命令展示】  Ruijie(config -vlan)#show vlan id 20  \nVLAN Name                             Status    Ports      \n---- --------------------------------  ---------  -----------------------------------  \n  20 VLAN0020                         STATIC    Gi0/1                               

In [22]:
1.35*30

40.5